In [10]:
import torch
import pandas as pd
import numpy as np
import csv
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaModel


In [11]:
# Carregar o tokenizer e modelo RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Carregar o dataset
df = pd.read_csv('../data_preprocessing/itens/dados_preprocessados.csv')

In [13]:
# Contar valores nulos por coluna
null_count = df.isnull().sum()

# Exibir a quantidade de valores nulos por coluna
print("Valores nulos por coluna:")
print(null_count)

Valores nulos por coluna:
page         0
url          0
issued       0
title      263
body         0
caption     83
dtype: int64


In [14]:
# Verificar linhas com valores nulos na coluna 'title'
null_title = df[df['title'].isnull()]

# Verificar linhas com valores nulos na coluna 'caption'
null_caption = df[df['caption'].isnull()]

# Exibir as linhas que possuem valores nulos nas colunas 'title' e 'caption'
print("Linhas com 'title' nulo:")
print(null_title)

print("\nLinhas com 'caption' nulo:")
print(null_caption)

Linhas com 'title' nulo:
                                        page  \
1906    53df5eff-aafc-4b86-859a-52d5008720c5   
2972    237eb885-c009-448d-80fb-69874c1a6d1f   
3228    d5a52ee9-dab2-4970-8491-b27ccdc8fa81   
3354    f11f12d7-ffb9-4b64-9542-348675ebbd51   
4021    4e4ca11a-1066-4f26-8619-0e1a256ff098   
...                                      ...   
251549  1b1e79e9-19ef-45cc-8292-a1de100e4c76   
252000  b33ddbb3-f5f5-418c-a35b-aef77a14679e   
252296  6a8ee504-36cb-455e-8e77-8022602853c9   
253096  2a5a41bf-a6d9-446e-91b6-85d7f8892065   
255501  88060942-f2e1-4f8d-954c-9213ade1736e   

                                                      url  \
1906    http://g1.globo.com/rs/rio-grande-do-sul/notic...   
2972    http://g1.globo.com/sp/campinas-regiao/terra-d...   
3228    http://g1.globo.com/pop-arte/musica/noticia/20...   
3354    http://g1.globo.com/am/amazonas/noticia/2021/0...   
4021    http://g1.globo.com/politica/noticia/2021/11/3...   
...                             

In [15]:
df_cleaned = df.dropna(axis=0, how='any')

In [16]:
null_count = df_cleaned.isnull().sum()
print("\nValores nulos por coluna após remoção de linhas com nulos:")
print(null_count)



Valores nulos por coluna após remoção de linhas com nulos:
page       0
url        0
issued     0
title      0
body       0
caption    0
dtype: int64


In [17]:
# Verificar se há GPU disponível e usar CPU caso contrário
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

save_path = '../processed/itens/embeddings/processed_data_embeddings.csv'  # Caminho de saída para os embeddings

# Garantir que as colunas 'title', 'body' e 'caption' são do tipo string
df['title'] = df['title'].fillna('').astype(str)
df['body'] = df['body'].fillna('').astype(str)
df['caption'] = df['caption'].fillna('').astype(str)

# Criar a pasta de saída, se não existir
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Função para verificar o ponto de interrupção e continuar de onde parou
def get_last_processed_row(save_path):
    if os.path.exists(save_path):
        # Ler o CSV para encontrar o número de linhas já processadas
        df_existing = pd.read_csv(save_path)
        return len(df_existing)  # Retorna o número de linhas já processadas
    return 0  # Caso o arquivo não exista, começa do início

# Função para gerar embeddings em batches
def generate_embeddings_in_batches_for_columns(df, column_name, batch_size=8, max_length=256, save_path=save_path):
    all_embeddings = []

    last_processed_row = get_last_processed_row(save_path)  # Pega a última linha processada

    # Inicializa o arquivo CSV e adiciona o cabeçalho (se ainda não existir)
    if last_processed_row == 0:
        with open(save_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['page', 'url', 'issued', column_name + '_embedding'])  # Cabeçalho para cada coluna

    # Processar os textos da coluna em lotes, a partir da última linha não processada
    texts = df[column_name].tolist()  # Selecionar a coluna
    for i in range(last_processed_row, len(texts)):
        batch = texts[i:i + batch_size]

        # Tokenizar o lote de textos
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

        # Gerar embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Média dos embeddings das palavras
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

        # Armazenar embeddings no arquivo CSV enquanto são gerados
        with open(save_path, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for idx, embedding in enumerate(embeddings):
                # Recuperar as outras colunas
                page = df.iloc[i + idx]['page']
                url = df.iloc[i + idx]['url']
                issued = df.iloc[i + idx]['issued']

                # Escrever no CSV com o embedding específico da coluna
                writer.writerow([page, url, issued, embedding.tolist()])  # Escreve as colunas originais e o embedding

        print(f"Processado batch {i // batch_size + 1} de {len(texts) // batch_size + 1}")

    return all_embeddings

# Gerar embeddings para as colunas 'title', 'body' e 'caption' separadamente
generate_embeddings_in_batches_for_columns(df, 'title', batch_size=8, max_length=256, save_path=save_path)
generate_embeddings_in_batches_for_columns(df, 'body', batch_size=8, max_length=256, save_path=save_path)
generate_embeddings_in_batches_for_columns(df, 'caption', batch_size=8, max_length=256, save_path=save_path)


[]